# 전처리


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# root path
root_path = '/content/drive/MyDrive/24YAICON'

# scene_name
scene_name = 'flowers_small'

In [12]:
import os
os.chdir(root_path)

### Git clone
* dust3r, 3DGS 클론

In [13]:
# !git clone https://github.com/graphdeco-inria/gaussian-splatting.git --recursive

In [14]:
# !git clone https://github.com/naver/dust3r --recursive

## Error 방지
###Colab Error
* 아래 코드로 안되면 python 환경변수 변경 필요

* 그래도 안되면 아래 참고:
https://medium.com/@tempmailwithpassword/how-to-fix-google-colabs-modulenotfounderror-400ba90043e7

In [15]:
import sys
sys.path.append(f"{root_path}/dust3r")
sys.path.append(f"{root_path}/gaussian-splatting")

In [16]:
sys.path.append(f"{root_path}/gaussian-splatting/submodules")

In [17]:
sys.path

['/content/drive/MyDrive/24YAICON/dust3r/croco',
 '/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor',
 '/content/drive/MyDrive/24YAICON/dust3r',
 '/content/drive/MyDrive/24YAICON/gaussian-splatting',
 '/content/drive/MyDrive/24YAICON/gaussian-splatting/submodules',
 '/content/drive/MyDrive/24YAICON/dust3r',
 '/content/drive/MyDrive/24YAICON/gaussian-splatting',
 '/content/drive/MyDrive/24YAICON/gaussian-splatting/submodules']

### Encoding Error


* wild-gs 파일 긁어올 때 가끔 인코딩오류 발생 -> 강제로 오버라이드

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### simple-knn 대체


In [34]:
file_path = os.path.join(root_path, "gaussian-splatting", "scene", "gaussian_model.py")

old_import = "from simple_knn._C import distCUDA2"

new_import = """\nfrom scipy.spatial import KDTree
import torch

def distCUDA2(points):
    points_np = points.detach().cpu().float().numpy()
    dists, inds = KDTree(points_np).query(points_np, k=4)
    meanDists = (dists[:, 1:] ** 2).mean(1)

    return torch.tensor(meanDists, dtype=points.dtype, device=points.device)
"""

with open(file_path, 'r') as file:
    file_data = file.read()

if old_import in file_data:
    file_data = file_data.replace(old_import, new_import)

    with open(file_path, 'w') as file:
        file.write(file_data)

    print(f"'{file_path}' 파일에서 '{old_import}'이(가) 성공적으로 대체되었습니다.")
else:
    print(f"'{file_path}' 파일에서 '{old_import}' 문구를 찾을 수 없습니다.")

'/content/drive/MyDrive/24YAICON/gaussian-splatting/scene/gaussian_model.py' 파일에서 'from simple_knn._C import distCUDA2' 문구를 찾을 수 없습니다.


## Installation
* dust3r ViT모델
* requirements.txt


### DUSt3R_ViTLarge_BaseDecoder_512

In [19]:
!wget -nc https://download.europe.naverlabs.com/ComputerVision/DUSt3R/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth -P checkpoints/

--2024-10-14 12:48:34--  https://download.europe.naverlabs.com/ComputerVision/DUSt3R/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
Resolving download.europe.naverlabs.com (download.europe.naverlabs.com)... 110.234.56.25
Connecting to download.europe.naverlabs.com (download.europe.naverlabs.com)|110.234.56.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2285005731 (2.1G)
Saving to: ‘checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth.2’

DUSt3R_ViTLarge_Bas 100%[===================>]   2.13G  32.0MB/s    in 76s     

2024-10-14 12:49:50 (28.8 MB/s) - ‘checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth.2’ saved [2285005731/2285005731]



### requirements install & upgrade

In [20]:
!pip install -r {root_path}/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10
ERROR: Could not find a version that satisfies the requirement waymo_open_dataset==1.0.1 (from versions: none)
ERROR: No matching distribution found for waymo_open_dataset==1.0.1


In [21]:
!pip install --upgrade setuptools wheel torch

## Dust3r

In [22]:
from dust3r.inference import inference
from dust3r.model import load_model
from dust3r.utils.image import load_images
from dust3r.utils.device import to_numpy
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

import torch
import numpy as np
import matplotlib.pyplot as plt

try:
    import lovely_tensors as lt
except:
    ! pip install --upgrade lovely-tensors
    import lovely_tensors as lt

lt.monkey_patch()

In [23]:
model_path = f"{root_path}/dust3r/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
device = 'cuda:0'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300

In [24]:
from pathlib import Path
Path.ls = lambda x: list(x.iterdir())

image_dir = Path(f"{root_path}/data/images/{scene_name}")

image_files = [str(x) for x in image_dir.ls() if x.suffix.lower() in ['.png', '.jpg']]
image_files = sorted(image_files, key=lambda x: int(x.split('DSC')[-1].split('.')[0])) # 인덱스 순으로 정렬

In [25]:
image_files

['/content/drive/MyDrive/24YAICON/data/images/flowers_small/_DSC9203.JPG',
 '/content/drive/MyDrive/24YAICON/data/images/flowers_small/_DSC9205.JPG',
 '/content/drive/MyDrive/24YAICON/data/images/flowers_small/_DSC9208.JPG']

In [26]:
model = load_model(model_path, device)
images = load_images(image_files, size=512)
pairs = make_pairs(images, scene_graph='complete', prefilter=None, symmetrize=True)
output = inference(pairs, model, device, batch_size=batch_size)

... loading model from /content/drive/MyDrive/24YAICON/dust3r/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth


/content/drive/MyDrive/24YAICON/dust3r/dust3r/model.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location='cpu')


instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>
>> Loading a list of 3 images
 - adding /content/drive/MyDrive/24YAICON/data/images/flowers_small/_DSC9203.JPG with resolution 5025x3312 --> 512x336
 - adding /content/drive/MyDrive/24YAICON/data/images/flowers_small/_DSC9205.JPG with resolution 5025x3312 --> 512x336
 - adding /content/drive/MyDrive/24YAICON/data/images/flowers_small/_DSC9208.JPG with resolution 5025x3312 --> 512x336
 (Found 3 images)
>> Inference with model on 6 image pairs


  0%|          | 0/6 [00:00<?, ?it/s]/content/drive/MyDrive/24YAICON/dust3r/dust3r/inference.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=bool(use_amp)):
/content/drive/MyDrive/24YAICON/dust3r/dust3r/model.py:205: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/content/drive/MyDrive/24YAICON/dust3r/dust3r/inference.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
100%|██████████| 6/6 [00:05<00:00,  1.06it/s]


In [27]:
scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer)
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

 init edge (0*,1*) score=33.26010513305664
 init edge (1,2*) score=2.7933757305145264
 init loss = 0.007108738180249929
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps', 'im_poses', 'im_focals']


100%|██████████| 300/300 [00:16<00:00, 17.89it/s, lr=1.27413e-06 loss=0.00187832]


# Construct colmap dataset

In [28]:
def inv(mat):
    """ Invert a torch or numpy matrix
    """
    if isinstance(mat, torch.Tensor):
        return torch.linalg.inv(mat)
    if isinstance(mat, np.ndarray):
        return np.linalg.inv(mat)
    raise ValueError(f'bad matrix type = {type(mat)}')

In [29]:
intrinsics = scene.get_intrinsics().detach().cpu().numpy()
world2cam = inv(scene.get_im_poses().detach()).cpu().numpy()
principal_points = scene.get_principal_points().detach().cpu().numpy()
focals = scene.get_focals().detach().cpu().numpy()
imgs = np.array(scene.imgs)
pts3d = [i.detach() for i in scene.get_pts3d()]
depth_maps = [i.detach() for i in scene.get_depthmaps()]

min_conf_thr = 20
scene.min_conf_thr = float(scene.conf_trf(torch.tensor(min_conf_thr)))
masks = to_numpy(scene.get_masks())

After convertion such data sctructure should appear

```
│   │   │   ├── images
│   │   │   ├── masks
│   │   │   ├── sparse/0
|   |   |   |    |------cameras.bin
|   |   |   |    |------images.bin
|   |   |   |    |------points3D.bin
|   |   |   |    |------points3D.ply
```

In [30]:
save_dir = Path(f"{root_path}/data/scenes/{scene_name}")
save_dir.mkdir(exist_ok=True, parents=True)

In [31]:
import os
from typing import NamedTuple, Optional
import cv2  # Assuming OpenCV is used for image saving
from scene.gaussian_model import BasicPointCloud
from PIL import Image
from scene.colmap_loader import rotmat2qvec
from utils.graphics_utils import focal2fov, fov2focal
from scene.dataset_readers import storePly

class CameraInfo(NamedTuple):
    uid: int
    R: np.ndarray
    T: np.ndarray
    FovY: np.ndarray
    FovX: np.ndarray
    image: np.ndarray
    image_path: str
    image_name: str
    width: int
    height: int
    mask: Optional[np.ndarray] = None
    mono_depth: Optional[np.ndarray] = None

class SceneInfo(NamedTuple):
    point_cloud: BasicPointCloud
    train_cameras: list
    test_cameras: list
    nerf_normalization: dict
    ply_path: str
    render_cameras: Optional[list[CameraInfo]] = None

def init_filestructure(save_path):
    save_path = Path(save_path)
    save_path.mkdir(exist_ok=True, parents=True)

    images_path = save_path / 'images'
    masks_path = save_path / 'masks'
    sparse_path = save_path / 'sparse/0'

    images_path.mkdir(exist_ok=True, parents=True)
    masks_path.mkdir(exist_ok=True, parents=True)
    sparse_path.mkdir(exist_ok=True, parents=True)

    return save_path, images_path, masks_path, sparse_path

def save_images_masks(imgs, masks, images_path, masks_path):
    # Saving images and optionally masks/depth maps
    for i, (image, mask) in enumerate(zip(imgs, masks)):
        image_save_path = images_path / f"{i}.png"

        mask_save_path = masks_path / f"{i}.png"
        image[~mask] = 1.
        rgb_image = cv2.cvtColor(image*255, cv2.COLOR_BGR2RGB)
        cv2.imwrite(str(image_save_path), rgb_image)

        mask = np.repeat(np.expand_dims(mask, -1), 3, axis=2)*255
        Image.fromarray(mask.astype(np.uint8)).save(mask_save_path)


def save_cameras(focals, principal_points, sparse_path, imgs_shape):
    # Save cameras.txt
    cameras_file = sparse_path / 'cameras.txt'
    with open(cameras_file, 'w') as cameras_file:
        cameras_file.write("# Camera list with one line of data per camera:\n")
        cameras_file.write("# CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]\n")
        for i, (focal, pp) in enumerate(zip(focals, principal_points)):
            cameras_file.write(f"{i} PINHOLE {imgs_shape[2]} {imgs_shape[1]} {focal[0]} {focal[0]} {pp[0]} {pp[1]}\n")

def save_imagestxt(world2cam, sparse_path):
     # Save images.txt
    images_file = sparse_path / 'images.txt'
    # Generate images.txt content
    with open(images_file, 'w') as images_file:
        images_file.write("# Image list with two lines of data per image:\n")
        images_file.write("# IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME\n")
        images_file.write("# POINTS2D[] as (X, Y, POINT3D_ID)\n")
        for i in range(world2cam.shape[0]):
            # Convert rotation matrix to quaternion
            rotation_matrix = world2cam[i, :3, :3]
            qw, qx, qy, qz = rotmat2qvec(rotation_matrix)
            tx, ty, tz = world2cam[i, :3, 3]
            images_file.write(f"{i} {qw} {qx} {qy} {qz} {tx} {ty} {tz} {i} {i}.png\n")
            images_file.write("\n") # Placeholder for points, assuming no points are associated with images here

import numpy as np
import os

def save_pointcloud_with_normals(imgs, pts3d, msk, sparse_path):
    pc = get_pc(imgs, pts3d, msk)  # Assuming get_pc is defined elsewhere and returns a trimesh point cloud

    # Define a default normal, e.g., [0, 1, 0]
    default_normal = [0, 1, 0]

    # Prepare vertices, colors, and normals for saving
    vertices = pc.vertices
    colors = pc.colors
    normals = np.tile(default_normal, (vertices.shape[0], 1))

    save_path = sparse_path / 'points3D.ply'

    # Construct the header of the PLY file
    header = """ply
format ascii 1.0
element vertex {}
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
property float nx
property float ny
property float nz
end_header
""".format(len(vertices))

    # Write the PLY file
    with open(save_path, 'w') as ply_file:
        ply_file.write(header)
        for vertex, color, normal in zip(vertices, colors, normals):
            ply_file.write('{} {} {} {} {} {} {} {} {}\n'.format(
                vertex[0], vertex[1], vertex[2],
                int(color[0]), int(color[1]), int(color[2]),
                normal[0], normal[1], normal[2]
            ))

import trimesh
def get_pc(imgs, pts3d, mask):
    imgs = to_numpy(imgs)
    pts3d = to_numpy(pts3d)
    mask = to_numpy(mask)

    pts = np.concatenate([p[m] for p, m in zip(pts3d, mask)])
    col = np.concatenate([p[m] for p, m in zip(imgs, mask)])

    pts = pts.reshape(-1, 3)[::3]
    col = col.reshape(-1, 3)[::3]

    #mock normals:
    normals = np.tile([0, 1, 0], (pts.shape[0], 1))

    pct = trimesh.PointCloud(pts, colors=col)
    pct.vertices_normal = normals  # Manually add normals to the point cloud

    return pct#, pts

def save_pointcloud(imgs, pts3d, msk, sparse_path):
    save_path = sparse_path / 'points3D.ply'
    pc = get_pc(imgs, pts3d, msk)

    pc.export(save_path)

In [32]:
save_path, images_path, masks_path, sparse_path = init_filestructure(save_dir)
save_images_masks(imgs, masks, images_path, masks_path)
save_cameras(focals, principal_points, sparse_path, imgs_shape=imgs.shape)
save_imagestxt(world2cam, sparse_path)
# save_pointcloud(imgs, pts3d, masks, sparse_path)
save_pointcloud_with_normals(imgs, pts3d, masks, sparse_path)

In [33]:
!nvcc --version
import torch
print(torch.version.cuda)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
12.1
